In [17]:
import os
import sys
import numpy as np
import pandas as pd
import cv2  # opencv+python == 4.9.0.80 # image processing
import matplotlib.pyplot as plt
import hdf5storage  # 0.1.19 # for loading .mat files

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import tensorflow as tf # Ensure using 2.13.0 as mentioned
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, Conv2DTranspose, BatchNormalization, Activation)
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers.legacy import Adam # trying legacy Adam for better performance
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator # optional data augmentation

import seaborn as sns

Algorithm for Class Balancing the Brain Tumor Dataset

- into the 3 planes: axial, coronal, sagittal
- where each tumor (meningioma, glioma, pituitary) has equal MRI scans for every plane

In [19]:
import os
import random
import h5py

# Define the path and number of files needed
base_path = '/Users/elizabethnemeti/Desktop/Planes'
target_count = 182
image_dimension = 512  # Adjust this to your actual image dimension if different

# Define the folders
folders = ['axial', 'sagittal', 'coronal']
tumor_types = {1: 'meningioma', 2: 'glioma', 3: 'pituitary'}

def get_tumor_type(mat_file_path):
    with h5py.File(mat_file_path, 'r') as mat_data:
        label = int(mat_data['label'][()])
    return tumor_types.get(label, 'unknown')

def ensure_equal_files(base_path, folders, tumor_types, target_count):
    for folder in folders:
        plane_path = os.path.join(base_path, folder)
        
        # Create a dictionary to hold file lists for each tumor type
        tumor_files = {tumor: [] for tumor in tumor_types.values()}
        
        for file_name in os.listdir(plane_path):
            if file_name.endswith('.mat'):
                file_path = os.path.join(plane_path, file_name)
                tumor_type = get_tumor_type(file_path)
                
                if tumor_type != 'unknown':
                    tumor_files[tumor_type].append(file_path)
        
        for tumor, files in tumor_files.items():
            current_count = len(files)
            
            if current_count > target_count:
                # If more than target, randomly remove files
                files_to_remove = random.sample(files, current_count - target_count)
                for file in files_to_remove:
                    os.remove(file)
            elif current_count < target_count:
                print(f"Not enough files for {tumor} in {folder}. Only {current_count} files found.")
                # If less than target, additional logic can be added here
                
            print(f"Processed {tumor} in {folder}: {min(current_count, target_count)} files.")

# Run the function
ensure_equal_files(base_path, folders, tumor_types, target_count)

KeyError: "Unable to open object (object 'label' doesn't exist)"